# Network Creation

In [1]:
import torch
from torch import nn
import matplotlib.pyplot as plt
from torch.distributions.categorical import Categorical

In [2]:
class ActorNetwork(nn.Module):
    """LSTM RNN for generating words for wordle solver"""
    def __init__(self, input_size, hidden_size, projection=1, num_layers=5, dropout_probability=0):
        super().__init__()
        self.network = nn.LSTM(input_size, hidden_size, num_layers, dropout=dropout_probability, proj_size=projection, batch_first=True)
        self.softmax = nn.Softmax(dim=1)
    def forward(self, x):
        output, (hidden, _) = self.network(x)
        probab = hidden + 1 / 2  # takes output of softmax [-1, 1] and squish to [0, 1]
        return probab 

class CriticNet(nn.Module):
    """Network representing the critic"""
    def __init__(self, in_shape):
        super().__init__()
        self.v_network = nn.Sequential(
            nn.Linear(in_shape, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
)

    def forward(self, x):
        return self.v_network(x)

# Toy Problem for RNN

In [3]:
net = ActorNetwork(1, 32, 26, 5)
input = torch.tensor([[
    [1],
    [2],
    [3],
    [4],
    [5]
]]).float()
print(input.shape)
net(input)

torch.Size([1, 5, 1])


tensor([[[0.7692, 0.5746, 0.4195, 0.4146, 0.4210, 0.3816, 0.3989, 0.5811,
          0.4722, 0.3840, 0.3775, 0.6665, 0.5746, 0.5107, 0.7378, 0.5991,
          0.4542, 0.3644, 0.3953, 0.5325, 0.3956, 0.3932, 0.4392, 0.5045,
          0.4452, 0.4789]],

        [[0.5529, 0.5420, 0.4617, 0.5714, 0.5202, 0.4860, 0.5416, 0.4653,
          0.5087, 0.5563, 0.4348, 0.4982, 0.5486, 0.4420, 0.5169, 0.4578,
          0.4986, 0.5326, 0.5001, 0.4297, 0.4435, 0.5217, 0.4514, 0.5539,
          0.5285, 0.4768]],

        [[0.4792, 0.4948, 0.5058, 0.4502, 0.5432, 0.4950, 0.4058, 0.5117,
          0.5769, 0.5007, 0.4828, 0.5071, 0.4394, 0.4662, 0.5303, 0.4848,
          0.5588, 0.5205, 0.5483, 0.4121, 0.4838, 0.4860, 0.5342, 0.4816,
          0.4495, 0.4791]],

        [[0.4992, 0.4535, 0.4806, 0.5109, 0.4697, 0.4390, 0.4651, 0.5697,
          0.4231, 0.4836, 0.4660, 0.4794, 0.5477, 0.5302, 0.5063, 0.4968,
          0.4790, 0.4739, 0.5020, 0.4796, 0.4986, 0.4689, 0.4918, 0.4997,
          0.4896, 0.5093]

# Figuring out the environment

In [4]:
import gym
import gym_wordle
wordle = gym.make("Wordle-v0")

In [5]:
wordle.reset()
obs = wordle.step([2, 17, 0, 19, 4])
wordle.render()
print(obs)
wordle.hidden_word

###################################################
c r a t e 

a b c d e f g h i j k l m n o p q r s t u v w x y z 
###################################################

({'board': array([[ 0,  0,  0,  2,  0],
       [-1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1]]), 'alphabet': array([ 0, -1,  0, -1,  0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        0, -1,  2, -1, -1, -1, -1, -1, -1])}, 0.0, False, {})


(24, 14, 20, 19, 7)

In [6]:
wordle.step([17, 4, 1, 0, 1])

({'board': array([[ 0,  0,  0,  2,  0],
         [ 0,  0,  0,  0,  0],
         [-1, -1, -1, -1, -1],
         [-1, -1, -1, -1, -1],
         [-1, -1, -1, -1, -1],
         [-1, -1, -1, -1, -1]]),
  'alphabet': array([ 0,  0,  0, -1,  0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
          0, -1,  2, -1, -1, -1, -1, -1, -1])},
 0.0,
 False,
 {})

In [7]:
import numpy as np
rnn = ActorNetwork(5, 32, 26, 5)
o = np.array([obs[0]['board']])
o = torch.from_numpy(o).to(torch.float32)
print(o.shape)
print(rnn(o))

torch.Size([1, 6, 5])
tensor([[[0.5784, 0.5548, 0.4521, 0.5546, 0.4620, 0.3900, 0.4581, 0.4895,
          0.3628, 0.6697, 0.5521, 0.5222, 0.5605, 0.5359, 0.5742, 0.4070,
          0.5257, 0.4200, 0.4461, 0.5238, 0.3695, 0.4998, 0.4732, 0.4871,
          0.5293, 0.4226]],

        [[0.5291, 0.4355, 0.4914, 0.5404, 0.5746, 0.4720, 0.4706, 0.4990,
          0.5314, 0.5156, 0.5034, 0.4374, 0.5129, 0.4606, 0.5518, 0.4645,
          0.4489, 0.5344, 0.5514, 0.4465, 0.4661, 0.4793, 0.4780, 0.5296,
          0.4005, 0.5002]],

        [[0.4930, 0.5260, 0.4909, 0.3779, 0.4686, 0.4809, 0.5326, 0.5496,
          0.5125, 0.5935, 0.4286, 0.5348, 0.4980, 0.4997, 0.4767, 0.5332,
          0.5729, 0.4449, 0.5696, 0.4686, 0.5495, 0.5180, 0.5247, 0.4759,
          0.4945, 0.4484]],

        [[0.5176, 0.4395, 0.4981, 0.4888, 0.5461, 0.5065, 0.5027, 0.4784,
          0.4654, 0.5807, 0.5061, 0.5163, 0.5095, 0.4966, 0.5762, 0.4662,
          0.4296, 0.5343, 0.4741, 0.5453, 0.5063, 0.4983, 0.4748, 0.4978,
   

In [8]:
prob = Categorical(rnn(o))
prob.sample()

tensor([[ 7],
        [ 8],
        [12],
        [ 4],
        [20]])

# Competing A2C solvers

In [9]:
def train(replay, q_val):
    vals = torch.vstack(replay.vals)
    
    rewards_tensor = torch.tensor(np.asarray(replay.rewards, dtype=np.float32))
    is_terminal_tensor = torch.tensor(np.asarray(replay.dones, dtype=np.int32))
        
    q_vals_tensor = rewards_tensor + discount_factor * q_val * (1 - is_terminal_tensor)
        
    advantage = q_vals_tensor - vals
    
    critic_loss = (advantage ** 2).mean()
    # critic_loss.requires_grad = True
    adam_critic.zero_grad()
    critic_loss.backward(retain_graph=True)
    adam_critic.step()
    
    
    log_probabs_tensor = torch.vstack(replay.log_probs)
    actor_loss = (-log_probabs_tensor*advantage.detach()).mean()
    # actor_loss.requires_grad = True
    adam_actor.zero_grad()
    actor_loss.backward()
    adam_actor.step()

In [35]:
class Advantage_ActorCritic():
    def __init__(self, world: gym.Env, policy_net: ActorNetwork, critic_net: CriticNet,
                 encoder, policy_alpha, critic_alpha, gamma, max_reward):
        # environment info
        self.world = world
        self.encoder = encoder
        self.max_reward = max_reward

        # actor and critic
        self.actor = policy_net
        self.critic = critic_net
        self.error_buffer = list()
        self.policy_optimizer = torch.optim.Adam(policy_net.network.parameters(), lr=policy_alpha)
        self.v_optimizer = torch.optim.Adam(critic_net.v_network.parameters(), lr=critic_alpha)

        # training info
        self.gamma = gamma
        self.episodes = 0

    def train(self, iterations):
        converged = False
        rewards = list()
        recents = torch.zeros(10)
        i = 0
        while not converged:
            r = self.episode()
            rewards.append(r)

            if i < 10:
                recents[i] = r
                i += 1

            else:
                recents.roll(-1, 0)
                recents[9] = r
            
            # convergence check
            if len(rewards) > 10 and recents.mean() >= self.max_reward:
                converged = True
            converged = True if iterations == self.episodes else False
        
        plt.plot(rewards)
        plt.show()


    def episode(self, training=True):
        done = False
        state = self.world.reset().copy()
        episode_reward = 0
        self.episodes += 1
        while not done:
            # take on policy action
            encoded = self.encoder(state)
            net_out = self.actor(encoded)

            dist = Categorical(net_out)
            action = dist.sample()
            try:
                state_prime, reward, done, _ = self.world.step(action)
            except AssertionError:
                reward = -.1
                state_prime = state.copy()

            # fill buffer
            self.error_buffer.append((state, state_prime, reward, dist.log_prob(action)))

            # prepare for next iteration
            episode_reward += reward
            state = state_prime.copy()
            if training:
                self.__net_update()

        return episode_reward

    def td_error(self, value, value_prime, reward):
        def loss_fn():
            print(value.shape, value_prime.shape) 
            return reward + self.gamma * value_prime - value
        return loss_fn()

    def policy_error(self, prob, error):
        def loss_fn():
            print(prob)
            print(error)
            return -torch.log(prob) * error
        return loss_fn()

    def __net_update(self):
        # calculate error for policy and critic
        self.critic.v_network.zero_grad()
        self.actor.network.zero_grad()

        state, state_prime, reward, prob = self.error_buffer.pop(0)
        print(self.encoder(state))
        value = self.critic(self.encoder(state))
        value_prime = self.critic(self.encoder(state_prime))

        td_error = self.td_error(value, value_prime, reward)
        policy_error = self.policy_error(prob, td_error) 

        # backprop
        if self.episodes > 100:
            td_error.backward()
        else:
            policy_error.backward()
            self.policy_optimizer.step()
        self.v_optimizer.step()

# Solving the Environment

In [36]:
critic = CriticNet(30)
actor = ActorNetwork(30, 32, 26)
agent = Advantage_ActorCritic(wordle, actor, critic, 
lambda x: torch.tensor([x['board']]).to(torch.float32).reshape(1, 1,30), 1e-3, 1e-3, 0.9, 1)

In [37]:
agent.episode()

tensor([[[-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
          -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
          -1., -1.]]])
torch.Size([1, 1, 1]) torch.Size([1, 1, 1])
tensor([[-3.1386],
        [-3.2099],
        [-3.1924],
        [-3.2840],
        [-3.2970]], grad_fn=<SqueezeBackward1>)
tensor([[[-0.0904]]], grad_fn=<SubBackward0>)


RuntimeError: grad can be implicitly created only for scalar outputs